In [1]:
import lib.load_data as ld
import lib.model as model
import lib.validation as validation
import os

**Fetch the dataset**

In [2]:

print("fetching dataset!") 
ld.download_mnist('dataset')
ld.download_fmnist('dataset')

fetching dataset!
creating  dataset/mnist


**Train the model**

In [3]:

print("training model!")  # replace this with code to train the model
latent_dim = 32
original_dim = (28, 28, 1)
batch_size = 100
epochs = 30
    
#load data
x_train, y_train, x_val, y_val, x_test, y_test = ld.load_mnist('dataset',flatten=False)
xf_train, yf_train, xf_test, yf_test, _, _ = ld.load_fmnist('dataset',flatten=False)

#encoder
encoder_inputs = model.keras.Input(shape=original_dim)
z_mean, z_log_var = model.create_encoder(encoder_inputs, latent_dim)
z = model.Sampling(z_mean, z_log_var)
encoder = model.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
#encoder.summary()

#decoder
latent_inputs = model.keras.Input(shape=(latent_dim,))
decoder_outputs = model.create_decoder(latent_inputs, latent_dim)
decoder = model.keras.Model(latent_inputs, decoder_outputs, name="decoder")
#decoder.summary()

vae = model.VAE(encoder, decoder)
vae.compile(optimizer=model.keras.optimizers.Adam())
vae.fit(x_train, epochs=epochs, batch_size=batch_size)
#vae.fit(xf_train, epochs=epochs, batch_size=batch_size)
  
#store model
if not (os.path.isdir('param')):
  os.makedirs('param')
vae.encoder.save("param/encoder_save", overwrite=True)
vae.decoder.save("param/decoder_save", overwrite=True)

training model!
Epoch 1/30
500/500 [==============================] - 36s 6ms/step - loss: 252.6548 - reconstruction_loss: 206.2020 - kl_loss: 5.2105
Epoch 2/30
500/500 [==============================] - 3s 6ms/step - loss: 186.2735 - reconstruction_loss: 176.9937 - kl_loss: 6.7532
Epoch 3/30
500/500 [==============================] - 3s 6ms/step - loss: 177.9383 - reconstruction_loss: 169.2544 - kl_loss: 7.1071
Epoch 4/30
500/500 [==============================] - 3s 6ms/step - loss: 172.4771 - reconstruction_loss: 164.4605 - kl_loss: 6.9015
Epoch 5/30
500/500 [==============================] - 3s 6ms/step - loss: 168.5942 - reconstruction_loss: 161.0883 - kl_loss: 6.7539
Epoch 6/30
500/500 [==============================] - 3s 6ms/step - loss: 165.7438 - reconstruction_loss: 158.4742 - kl_loss: 6.7936
Epoch 7/30
500/500 [==============================] - 3s 6ms/step - loss: 163.5702 - reconstruction_loss: 156.5498 - kl_loss: 6.8173
Epoch 8/30
500/500 [==============================] 

**Evaluate model and compute metrics and figures**

In [4]:

print("evaluating model!")
t = -500
#load model
encoder = validation.keras.models.load_model("param/encoder_save")
decoder = validation.keras.models.load_model("param/decoder_save")
vae = model.VAE(encoder, decoder)
vae.compile(optimizer=model.keras.optimizers.Adam())

#load data
_, _, _, _, x_test, y_test = ld.load_mnist('dataset',flatten=False)
_, _, xf_test, yf_test, _, _ = ld.load_fmnist('dataset',flatten=False)
xn_test, yn_test = ld.load_mnist_vs_fmnist('dataset',flatten=False)

validation.plot_hist(x_in=x_test, x_out=xf_test, vae=vae)
validation.plot_norm(x_in=x_test, x_out=xf_test, vae=vae)
#validation.plot_hist(x_in=xf_test, x_out=x_test, vae=vae)
#validation.plot_norm(x_in=xf_test, x_out=x_test, vae=vae)

#validation.VA_spec(xf_test,yf_test)

print(validation.accurcy (xn_test,vae, yn_test, t))

evaluating model!
0.878
